<a href="https://colab.research.google.com/github/eujin99/real-estate-support-dataset/blob/main/hoon_notion_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets huggingface_hub

In [ ]:
from google.colab import drive

# Google Drive 마운트
drive.mount('/content/drive')


In [ ]:
# CSV 파일 경로 설정 (Google Drive의 파일 경로 기재)
csv_file_path = '/content/drive/MyDrive/real_estate_dataset/hoon.csv'


In [ ]:
from huggingface_hub import notebook_login

notebook_login()


In [ ]:
from datasets import load_dataset, Dataset

# CSV 파일을 로드 -> 데이터셋 변환
dataset = Dataset.from_csv(csv_file_path)


In [ ]:
# 새로운 데이터셋 만들었슴..
dataset.push_to_hub("chloestella/hoon_dataset")


In [ ]:
import pandas as pd
import random
import json

# CSV 파일 경로 설정
csv_file_path = '/content/drive/MyDrive/real_estate_dataset/hoon.csv'

# CSV 파일 로드
df = pd.read_csv(csv_file_path)

# 잘뮨-답변 쌍 데이터셋 만들어주기 (임시) 나중에 더 추가
def generate_qa_pairs(df):
    qa_pairs = []

    for _, row in df.iterrows():
        # 매매가
        if pd.notnull(row['매매가']):
            price_patterns = [
                f"{row['매매가']}만원 이하의 매물을 추천해줘.",
                f"{row['매매가']}만원 이하로 살 수 있는 집이 있나요?",
                f"매매가 {row['매매가']}만원 이하인 집을 찾아줘."
            ]
            question_price = random.choice(price_patterns)
            answer_price = f"{row['주소']}에 위치한 {row['현장명']}은 매매가 {row['매매가']}만원, 평수 {row['평수']}제곱미터입니다."
            qa_pairs.append({'question': question_price, 'answer': answer_price})

        # 층수
        if pd.notnull(row['층수']):
            floor_patterns = [
                f"{row['층수']}층 이상의 매물을 찾아줘.",
                f"{row['층수']}층보다 높은 매물이 있나요?",
                f"최소 {row['층수']}층인 집을 찾고 있어."
            ]
            question_floor = random.choice(floor_patterns)
            answer_floor = f"{row['주소']}에 위치한 {row['현장명']}은 {row['층수']}층이며, 매매가는 {row['매매가']}만원입니다."
            qa_pairs.append({'question': question_floor, 'answer': answer_floor})

        # 실입
        if pd.notnull(row['실입']):
            deposit_patterns = [
                f"{row['실입']}만원 이하의 실입으로 가능한 매물을 추천해줘.",
                f"실입이 {row['실입']}만원 이하인 집을 찾고 있어.",
                f"실입금 {row['실입']}만원 이하의 매물은?"
            ]
            question_deposit = random.choice(deposit_patterns)
            answer_deposit = f"{row['주소']}에 위치한 {row['현장명']}은 실입 {row['실입']}만원으로 가능합니다."
            qa_pairs.append({'question': question_deposit, 'answer': answer_deposit})

        # 거실창
        if pd.notnull(row['거실창']) and '좋음' in str(row['거실창']):
            livingroom_patterns = [
                "거실창이 큰 곳을 추천해줘.",
                "거실창이 좋은 집을 찾아줘.",
                "거실창이 좋은 매물을 찾고 있어."
            ]
            question_livingroom = random.choice(livingroom_patterns)
            answer_livingroom = f"{row['주소']}에 위치한 {row['현장명']}은 거실창이 좋음으로 평가되어 있습니다."
            qa_pairs.append({'question': question_livingroom, 'answer': answer_livingroom})

        # 역세권
        if pd.notnull(row['구조,위치 특징']) and '역세권' in str(row['구조,위치 특징']):
            station_patterns = [
                "역세권 매물을 추천해줘.",
                "지하철역 가까운 집을 찾아줘.",
                "역세권에 위치한 매물이 있나요?"
            ]
            question_station = random.choice(station_patterns)
            answer_station = f"{row['주소']}에 위치한 {row['현장명']}은 역세권에 있으며, 매매가는 {row['매매가']}만원입니다."
            qa_pairs.append({'question': question_station, 'answer': answer_station})

        # 특이사항
        if pd.notnull(row['특이사항']):
            special_patterns = [
                "특이사항이 있는 매물을 추천해줘.",
                "특별한 조건이 있는 집을 찾아줘.",
                "특이사항이 있는 매물은?"
            ]
            question_special = random.choice(special_patterns)
            answer_special = f"{row['주소']}에 위치한 {row['현장명']}은 다음과 같은 특이사항이 있습니다: {row['특이사항']}."
            qa_pairs.append({'question': question_special, 'answer': answer_special})

    return qa_pairs

# 질문-답변 쌍 생성
qa_pairs = generate_qa_pairs(df)

# 질문-답변 데이터 -> CSV 저장
qa_df = pd.DataFrame(qa_pairs)
qa_df.to_csv('/content/drive/MyDrive/real_estate_dataset/qa_data.csv', index=False, encoding='utf-8-sig')
print("질문-답변 데이터가 CSV 파일로 저장됨.")

# 혹시나 일단 JSON으로도 저장해두기
with open('/content/drive/MyDrive/real_estate_dataset/qa_data.json', 'w', encoding='utf-8') as f:
    json.dump(qa_pairs, f, ensure_ascii=False, indent=4)
print("질문-답변 데이터가 JSON 파일로 저장됨.")


In [ ]:
import pandas as pd

# CSV 파일 경로
qa_data_path = '/content/drive/MyDrive/real_estate_dataset/qa_data.csv'

# CSV 파일 로드
qa_data = pd.read_csv(qa_data_path)

# 출력 테스트 해보깅,,
pd.set_option('display.max_rows', None)
print(qa_data)


In [ ]:
from datasets import Dataset

# CSV 파일 경로 설정
csv_file_path = '/content/drive/MyDrive/real_estate_dataset/qa_data.csv'

# CSV 파일 로드
dataset = Dataset.from_csv(csv_file_path)

# Hugging Face Hub에 업로드
dataset.push_to_hub("chloestella/qa_data")


In [ ]:
from datasets import Dataset

# CSV 파일 경로 설정
csv_file_path = '/content/drive/MyDrive/real_estate_dataset/qa_data.csv'

# CSV 파일 로드하여 Dataset 객체 생성
dataset = Dataset.from_csv(csv_file_path)

# Hugging Face Hub에 업로드
dataset.push_to_hub("chloestella/hoon_qa")
